# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [39]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [40]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jamestown,42.0970,-79.2353,-5.20,73,100,4.63,US,Jamestown
1,1,mar del plata,-38.0023,-57.5575,19.91,59,0,6.69,AR,Mar del Plata
2,2,nar'yan-mar,67.6713,53.0870,-7.52,94,100,5.61,RU,Naryan-Mar
3,3,hilo,19.7297,-155.0900,25.61,78,33,0.45,US,Hilo
4,4,recreio,-21.5250,-42.4692,24.70,81,63,1.30,BR,Recreio


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [41]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    scale = .5)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [42]:
# Narrow down cities that fit criteria and drop any results with null values

vac_city_df = city_data_df[((city_data_df['Max Temp'] > 21) &
                             (city_data_df['Max Temp'] < 27) &
                             (city_data_df['Wind Speed'] <4.5) &
                             (city_data_df['Cloudiness'] == 0))]

# Drop any rows with null values
clean_data = vac_city_df.dropna()

# Display sample data
clean_data


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
108,108,nova vicosa,-17.8919,-39.3719,25.05,85,0,4.16,BR,Nova Viçosa
254,254,saint-pierre,-21.3393,55.4781,25.82,89,0,2.57,RE,Saint-Pierre
279,279,isla del bosque,22.7333,-105.8500,23.28,45,0,4.47,MX,Isla del Bosque
358,358,villamontes,-21.2500,-63.5000,26.27,34,0,0.89,BO,Villamontes
466,466,chillan,-36.6066,-72.1034,23.18,35,0,4.12,CL,Chillán
558,558,silver city,32.7701,-108.2803,21.48,12,0,4.12,US,Silver City
559,559,guacamayas,18.0167,-102.2167,26.88,64,0,2.62,MX,Guacamayas


### Step 3: Create a new DataFrame called `hotel_df`.

In [43]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_data[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ' '

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
108,nova vicosa,BR,-17.8919,-39.3719,85,
254,saint-pierre,RE,-21.3393,55.4781,89,
279,isla del bosque,MX,22.7333,-105.8500,45,
358,villamontes,BO,-21.2500,-63.5000,34,
466,chillan,CL,-36.6066,-72.1034,35,
558,silver city,US,32.7701,-108.2803,12,
559,guacamayas,MX,18.0167,-102.2167,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [45]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = 'accomodation.hotel'

params = {
    'limit':limit,
    'apiKey':geoapify_key,
    'categories':categories
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df.loc[index, 'Lat']
    longitude=hotel_df.loc[index, 'Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude}, {latitude}, {radius}'
    params["bias"] = f'proximity:{longitude}, {latitude}'
    params['categories'] = 'accomodation.hotel'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address_json = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nova vicosa - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
isla del bosque - nearest hotel: No hotel found
villamontes - nearest hotel: No hotel found
chillan - nearest hotel: No hotel found
silver city - nearest hotel: No hotel found
guacamayas - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
108,nova vicosa,BR,-17.8919,-39.3719,85,No hotel found
254,saint-pierre,RE,-21.3393,55.4781,89,No hotel found
279,isla del bosque,MX,22.7333,-105.8500,45,No hotel found
358,villamontes,BO,-21.2500,-63.5000,34,No hotel found
466,chillan,CL,-36.6066,-72.1034,35,No hotel found
558,silver city,US,32.7701,-108.2803,12,No hotel found
559,guacamayas,MX,18.0167,-102.2167,64,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)